# 2015 Flight Delays and Cancellations
Which airline should you fly on to avoid significant delays?

https://www.kaggle.com/usdot/flight-delays

In [1]:
import pandas as pd

In [2]:
pd.options.display.max_rows = 5
pd.options.display.max_columns = 50

In [17]:
import numpy as np, time
from sklearn.model_selection import train_test_split

data = pd.read_csv("data/flights.csv", nrows=70000)
# data = data.sample(frac = 0.1, random_state=10)



In [18]:
print("data shape: ", data.shape)

data.head()

data shape:  (70000, 31)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]
data

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DESTINATION_AIRPORT,ORIGIN_AIRPORT,AIR_TIME,DEPARTURE_TIME,DISTANCE,ARRIVAL_DELAY
0,1,1,4,AS,98,SEA,ANC,169.0,2354.0,1448,-22.0
1,1,1,4,AA,2336,PBI,LAX,263.0,2.0,2330,-9.0
2,1,1,4,US,840,CLT,SFO,266.0,18.0,2296,5.0
3,1,1,4,AA,258,MIA,LAX,258.0,15.0,2342,-9.0
4,1,1,4,AS,135,ANC,SEA,199.0,24.0,1448,-21.0
...,...,...,...,...,...,...,...,...,...,...,...
69995,1,5,1,WN,1651,DEN,PHX,84.0,1327.0,602,39.0
69996,1,5,1,WN,3918,MSP,PHX,172.0,1236.0,1276,-2.0
69997,1,5,1,WN,376,LAS,SAT,148.0,1306.0,1069,7.0
69998,1,5,1,WN,2227,PHX,MCI,146.0,1246.0,1044,-4.0


In [20]:
pd.options.display.max_rows = 20
data.isnull().sum()

MONTH                     0
DAY                       0
DAY_OF_WEEK               0
AIRLINE                   0
FLIGHT_NUMBER             0
DESTINATION_AIRPORT       0
ORIGIN_AIRPORT            0
AIR_TIME               1763
DEPARTURE_TIME         1580
DISTANCE                  0
ARRIVAL_DELAY          1763
dtype: int64

In [21]:
data.dropna(inplace=True)

In [22]:
data.isnull().sum()

MONTH                  0
DAY                    0
DAY_OF_WEEK            0
AIRLINE                0
FLIGHT_NUMBER          0
DESTINATION_AIRPORT    0
ORIGIN_AIRPORT         0
AIR_TIME               0
DEPARTURE_TIME         0
DISTANCE               0
ARRIVAL_DELAY          0
dtype: int64

In [23]:
data["ARRIVAL_DELAY"].describe()

count    68237.000000
mean        17.760262
std         48.017738
min        -65.000000
25%         -8.000000
50%          4.000000
75%         26.000000
max       1384.000000
Name: ARRIVAL_DELAY, dtype: float64

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68237 entries, 0 to 69999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MONTH                68237 non-null  int64  
 1   DAY                  68237 non-null  int64  
 2   DAY_OF_WEEK          68237 non-null  int64  
 3   AIRLINE              68237 non-null  object 
 4   FLIGHT_NUMBER        68237 non-null  int64  
 5   DESTINATION_AIRPORT  68237 non-null  object 
 6   ORIGIN_AIRPORT       68237 non-null  object 
 7   AIR_TIME             68237 non-null  float64
 8   DEPARTURE_TIME       68237 non-null  float64
 9   DISTANCE             68237 non-null  int64  
 10  ARRIVAL_DELAY        68237 non-null  float64
dtypes: float64(3), int64(5), object(3)
memory usage: 6.2+ MB


In [25]:
data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1
data["ARRIVAL_DELAY"].value_counts()


0    40749
1    27488
Name: ARRIVAL_DELAY, dtype: int64

In [26]:
cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1
 

                                      

In [27]:
data.AIRLINE.value_counts()

14    14527
4      9095
5      7195
10     7115
1      6495
11     5831
12     4913
8      3769
3      3311
2      2023
9      1256
6      1036
7       935
13      736
Name: AIRLINE, dtype: int64

In [28]:
train, test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"],
                                                random_state=10, test_size=0.25)


## XGBoost

In [29]:
%%time 
import xgboost as xgb
from sklearn import metrics

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

# Parameter Tuning
model = xgb.XGBClassifier()
# param_dist = {"max_depth": [10,30,50],
#               "min_child_weight" : [1,3,6],
#               "n_estimators": [200],
#               "learning_rate": [0.05, 0.1,0.16],}
# grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
#                                    verbose=10, n_jobs=-1)
# grid_search.fit(train, y_train)

# grid_search.best_estimator_

model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(train,y_train)

auc(model, train, test)

[01:15:50] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


CPU times: user 54.2 s, sys: 111 ms, total: 54.3 s
Wall time: 54.3 s


(1.0, 0.8091628449652525)

## Light GBM

In [ ]:
%%time
import lightgbm as lgb
from sklearn import metrics

def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
                            metrics.roc_auc_score(y_test,m.predict(test)))

lg = lgb.LGBMClassifier(silent=False)
# param_dist = {"max_depth": [25,50, 75],
#               "learning_rate" : [0.01,0.05,0.1],
#               "num_leaves": [300,900,1200],
#               "n_estimators": [200]
#              }
# grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
# grid_search.fit(train,y_train)
# grid_search.best_estimator_

d_train = lgb.Dataset(train, label=y_train)
params = {"max_depth": 50, "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 300}

# Without Categorical Features
model2 = lgb.train(params, d_train)
auc2(model2, train, test)

#With Catgeorical Features
cate_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT"]
model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)
auc2(model2, train, test)


## Catboost

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51177 entries, 936 to 51777
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MONTH                51177 non-null  int64  
 1   DAY                  51177 non-null  int64  
 2   DAY_OF_WEEK          51177 non-null  int64  
 3   AIRLINE              51177 non-null  int8   
 4   FLIGHT_NUMBER        51177 non-null  int16  
 5   DESTINATION_AIRPORT  51177 non-null  int16  
 6   ORIGIN_AIRPORT       51177 non-null  int16  
 7   AIR_TIME             51177 non-null  float64
 8   DEPARTURE_TIME       51177 non-null  float64
 9   DISTANCE             51177 non-null  int64  
dtypes: float64(2), int16(3), int64(4), int8(1)
memory usage: 3.1 MB


In [32]:
%%time
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

# params = {'depth': [4, 7, 10],
#           'learning_rate' : [0.03, 0.1, 0.15],
#          'l2_leaf_reg': [1,4,9],
#          'iterations': [300]}
# cb = cb.CatBoostClassifier()
# cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
# cb_model.fit(train, y_train)

# With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, 
                            iterations= 500, l2_leaf_reg= 9, 
                            learning_rate= 0.15)
clf.fit(train,y_train)


# With Categorical features
clf2 = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
                            depth=10, iterations= 500, l2_leaf_reg= 9, 
                             learning_rate= 0.15)
clf2.fit(train,y_train, cat_features= cat_features_index)
print("No cat_features_index ")
print(auc(clf, train, test))
print("With cat_features_index ")
print(auc(clf2, train, test))

0:	total: 67.7ms	remaining: 33.8s
1:	total: 131ms	remaining: 32.7s
2:	total: 195ms	remaining: 32.3s
3:	total: 262ms	remaining: 32.5s
4:	total: 324ms	remaining: 32.1s
5:	total: 391ms	remaining: 32.2s
6:	total: 463ms	remaining: 32.6s
7:	total: 526ms	remaining: 32.3s
8:	total: 584ms	remaining: 31.9s
9:	total: 653ms	remaining: 32s
10:	total: 717ms	remaining: 31.9s
11:	total: 780ms	remaining: 31.7s
12:	total: 843ms	remaining: 31.6s
13:	total: 896ms	remaining: 31.1s
14:	total: 963ms	remaining: 31.2s
15:	total: 1.03s	remaining: 31.2s
16:	total: 1.09s	remaining: 31s
17:	total: 1.16s	remaining: 31.1s
18:	total: 1.23s	remaining: 31s
19:	total: 1.28s	remaining: 30.8s
20:	total: 1.35s	remaining: 30.7s
21:	total: 1.42s	remaining: 30.8s
22:	total: 1.48s	remaining: 30.7s
23:	total: 1.55s	remaining: 30.7s
24:	total: 1.61s	remaining: 30.7s
25:	total: 1.68s	remaining: 30.6s
26:	total: 1.74s	remaining: 30.4s
27:	total: 1.8s	remaining: 30.3s
28:	total: 1.85s	remaining: 30.1s
29:	total: 1.91s	remaining: 29